In [51]:
# DF 
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from math import floor



# visualization 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

# time series
import yfinance as yf
from statsmodels.tsa.stattools import adfuller, kpss # Tests for stationarity
from statsmodels.tsa.statespace.sarimax import SARIMAX # Actual Model
# from pmdarima.arima import auto_arima
# import pmdarima
from statsmodels.tsa.seasonal import seasonal_decompose # Decomposing into trend, seasonal, residual
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # Plotting (Partial) Autocorrelation
from statsmodels.tsa.stattools import acf, pacf # Calculating (Partial) Autocorrelation Functions
from scipy.stats import normaltest, boxcox # Tests if data came from a normal distribution, is a transformation

# Evaluation
from sklearn.metrics import mean_squared_error # A metric for evaluating a model
from sklearn.model_selection import train_test_split

# To ensure reproducability
import random

# To see progress
from tqdm import tqdm

# pipelines and regressions
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score


### Loading the data

In [30]:
stocks = pd.read_csv('sp500.csv')
stocks.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [3]:
idx = 85
ticker, company = stocks['Symbol'].iloc[idx], stocks['Security'].iloc[idx]
print(ticker, company)

COF Capital One


In [31]:
start_date = '2021-01-01'
end_date = date.today()

In [32]:
df = yf.download(ticker, start = start_date, end = end_date)
df
df.to_csv('data.csv')
df = pd.read_csv('data.csv')
df

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,99.730003,99.989998,96.980003,97.839996,91.281059,3429700
1,2021-01-05,97.790001,100.580002,97.790001,100.139999,93.426872,2171500
2,2021-01-06,103.000000,107.650002,102.559998,107.379997,100.181511,4394000
3,2021-01-07,108.650002,111.540001,108.000000,110.730003,103.306946,4795700
4,2021-01-08,111.000000,111.779999,108.639999,110.559998,103.148331,3001100
...,...,...,...,...,...,...,...
868,2024-06-17,133.789993,137.350006,133.500000,137.100006,137.100006,2054500
869,2024-06-18,137.009995,138.210007,136.770004,137.339996,137.339996,1399600
870,2024-06-20,137.820007,138.149994,135.839996,138.130005,138.130005,1811800
871,2024-06-21,137.000000,137.770004,135.210007,136.770004,136.770004,3152700


In [33]:
df['Daily Returns'] = df['Close'].pct_change().fillna(0)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Returns
0,2021-01-04,99.730003,99.989998,96.980003,97.839996,91.281059,3429700,0.000000
1,2021-01-05,97.790001,100.580002,97.790001,100.139999,93.426872,2171500,0.023508
2,2021-01-06,103.000000,107.650002,102.559998,107.379997,100.181511,4394000,0.072299
3,2021-01-07,108.650002,111.540001,108.000000,110.730003,103.306946,4795700,0.031198
4,2021-01-08,111.000000,111.779999,108.639999,110.559998,103.148331,3001100,-0.001535


In [34]:
fig1 = px.histogram(df, x='Daily Returns', text_auto=True).update_layout(title=f"Daily Returns for {company}", yaxis_title='Frequency')
fig1.show()

### Feature Engineering

Important features in making a predictive model for financial data:
1. RSI: Relative Strength Index (measures the speed at which prices change; determines if its overbought or oversold)
2. MACD: Moving Average Convergence Distance (momentum oscillator to help identify potential buying/selling opportunities)

In [35]:
# calculating 14 day rsi
delta = df['Close'].diff()
gain= delta.copy()
loss = delta.copy()
gain[gain<0] = 0
loss[loss > 0] = 0
avg_gain = gain.rolling(window=14).mean()
avg_loss = np.abs(loss.rolling(window=14).mean())
rs = avg_gain/avg_loss
rsi = 100 - (100/(1+rs))

In [36]:
# MACD  
ema_12 = df['Close'].ewm(span=12, adjust=False).mean() #ema = estimated moving average, ewm = exponentially weighted moving average
ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
macd = ema_26-ema_12

In [37]:
df['RSI'] = rsi
df['MACD']= macd
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Returns,RSI,MACD
868,2024-06-17,133.789993,137.350006,133.500000,137.100006,137.100006,2054500,0.025430,50.887593,1.333713
869,2024-06-18,137.009995,138.210007,136.770004,137.339996,137.339996,1399600,0.001750,53.211708,1.186720
870,2024-06-20,137.820007,138.149994,135.839996,138.130005,138.130005,1811800,0.005752,53.669570,0.995010
871,2024-06-21,137.000000,137.770004,135.210007,136.770004,136.770004,3152700,-0.009846,48.376133,0.941961
872,2024-06-24,137.039993,139.110001,135.630005,137.679993,137.679993,1749000,0.006653,51.041662,0.817072


### Model Training

In [38]:
X = df[['RSI', 'MACD']]
y = np.where(df['Close'].shift(-1) > df['Close'], 1, -1)

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train in histogram gradient boosting classifier model
model = HistGradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 0.4685714285714286


### Model Evaluation

In [39]:
df['Predicted Signal'] = model.predict(X)
df['Strategy Returns'] = df['Daily Returns']*df['Predicted Signal'].shift()
df['Cumulative Returns'] = (1+df['Strategy Returns']).cumprod()

In [40]:
fig2 = px.line(df, x='Date', y='Cumulative Returns').update_layout(title = f'Cumulative Returns of the Trading Strategy for {company}')
fig2

## Tests for stationarity

In [41]:
df1 = df[['Date', 'Adj Close']]
df1.head()

,Date,Adj Close
0,2021-01-04,91.281059
1,2021-01-05,93.426872
2,2021-01-06,100.181511
3,2021-01-07,103.306946
4,2021-01-08,103.148331


In [42]:
train_split = floor(0.8*df['Adj Close'].size)
y_train, y_test = df1['Adj Close'].iloc[:train_split], df1['Adj Close'].iloc[train_split:]

### Visualization

In [43]:
# line plot
fig3 = px.line(df1, x='Date', y='Adj Close')
fig3.update_layout(title = f'Stock Price for {company}')
fig3

In [44]:
# The Augmented Dickey-Fuller Test
# Statistical hypothesis test that helps assess stationarity. Null hypothesis: Time series is not stationary

def adf_results(time_series):
    # Use the statsmodels implementation of ADF test
    result = adfuller(time_series)

    # Print results
    print('ADF Statistic: {}\np-value: {}\nCritical Values: {}'.format(result[0], result[1], result[4]))

    # Based on p-value, can determine if stationary
    stationary = (result[1] <= 0.05)
    
    # Conditionally adds the word not
    print('Result is stationary' if stationary else 'Result is NOT stationary')

In [45]:
adf_results(y_train)


ADF Statistic: -1.6398405678934016
p-value: 0.46234410196384323
Critical Values: {'1%': np.float64(-3.439766853257416), '5%': np.float64(-2.8656956054873377), '10%': np.float64(-2.5689829557089308)}
Result is NOT stationary


In [46]:
kpss(y_train)


C:\Users\namitha vishnupad\AppData\Local\Temp\ipykernel_5000\2798389234.py:1: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




(np.float64(2.2917370245805953),
 np.float64(0.01),
 17,
 {'10%': 0.347, '5%': 0.463, '2.5%': 0.574, '1%': 0.739})

In [47]:
diffs = y_train.diff().dropna()
# # Line plot
fig4 = px.line(diffs)
fig4

# Title
fig4.update_layout(title=f"Change in Stock Price for {company}")

# Display the plot
fig4.show()

In [48]:
adf_results(diffs)

ADF Statistic: -25.84976710226118
p-value: 0.0
Critical Values: {'1%': np.float64(-3.4397804336105198), '5%': np.float64(-2.865701589065464), '10%': np.float64(-2.5689861435625576)}
Result is stationary


In [49]:
kpss(diffs)


C:\Users\namitha vishnupad\AppData\Local\Temp\ipykernel_5000\1752755275.py:1: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.




(np.float64(0.2334753712743753),
 np.float64(0.1),
 2,
 {'10%': 0.347, '5%': 0.463, '2.5%': 0.574, '1%': 0.739})